In [ ]:
pip install pycaret[models]

In [2]:
# check installed version
import pycaret
pycaret.__version__

'3.3.1'

In [3]:
from pycaret.classification import *

In [46]:
import pandas as pd

In [106]:
# df=pd.read_csv("/content/main_AAPL_data.csv")
# df=pd.read_csv("/content/main_AMZN_data.csv")
# df=pd.read_csv("/content/main_GOOGL_data.csv")
# df=pd.read_csv("/content/main_META_data.csv")
# df=pd.read_csv("/content/main_MSFT_data.csv")
# df=pd.read_csv("/content/main_NVDA_data.csv")
df=pd.read_csv("/content/main_TSLA_data.csv")

df.head()

,Date,Date.1,Open,y_label,OperatingCashFlow,\tCashFlowFromContinuingOperatingActivities,\t\tNetIncomeFromContinuingOperations,\t\tOperatingGainsLosses,\t\t\tGainLossOnSaleOfPPE,\t\t\tNetForeignCurrencyExchangeGainLoss,...,CapitalLeaseObligations,NetTangibleAssets,WorkingCapital,InvestedCapital,TangibleBookValue,TotalDebt,NetDebt,ShareIssued,OrdinarySharesNumber,TreasurySharesNumber
0,12/31/2023,03/01/24,200.520004,1,"4,37,00,00,000","4,37,00,00,000","7,94,30,00,000","17,30,00,000",0,"17,30,00,000",...,"4,91,60,00,000","62,01,90,00,000","20,86,80,00,000","67,29,10,00,000","62,01,90,00,000","9,57,30,00,000",NaN,"3,18,50,00,000","3,18,50,00,000",0.0
1,9/30/2023,12/01/23,233.139999,0,"3,30,80,00,000","3,30,80,00,000","1,87,80,00,000","-3,40,00,000",0,"-3,40,00,000",...,"4,48,30,00,000","52,84,10,00,000","18,38,60,00,000","57,17,00,00,000","52,84,10,00,000","8,18,70,00,000",NaN,"3,17,90,00,000","3,17,90,00,000",0.0
2,6/30/2023,09/01/23,257.260010,0,"3,06,50,00,000","3,06,50,00,000","2,61,40,00,000","-25,80,00,000",NaN,"-25,80,00,000",...,"4,29,20,00,000","50,48,10,00,000","16,28,30,00,000","52,64,90,00,000","50,48,10,00,000","5,81,10,00,000",NaN,"3,17,40,00,000","3,17,40,00,000",NaN
3,3/31/2023,06/01/23,202.589996,0,"2,51,30,00,000","2,51,30,00,000","2,53,90,00,000","-2,50,00,000",NaN,"-2,50,00,000",...,"3,82,20,00,000","47,47,10,00,000","15,56,10,00,000","49,80,60,00,000","47,47,10,00,000","5,57,40,00,000",NaN,"3,16,90,00,000","3,16,90,00,000",NaN
4,12/31/2022,03/01/23,206.210007,0,"3,27,80,00,000","3,27,80,00,000","3,70,70,00,000","11,40,00,000","3,40,00,000","8,00,00,000",...,"3,70,30,00,000","44,11,10,00,000","14,20,80,00,000","46,74,90,00,000","44,11,10,00,000","5,74,80,00,000",NaN,"3,16,40,00,000","3,16,40,00,000",NaN


In [107]:
df = df.loc[:,~df.columns.str.contains('Date', case=False)]
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace(',', '').astype(float)
df = df.dropna(axis=1)
df.columns = [col.replace('\t', '') for col in df.columns]
df.head()

,Open,y_label,OperatingCashFlow,CashFlowFromContinuingOperatingActivities,NetIncomeFromContinuingOperations,DepreciationAmortizationDepletion,DepreciationAndAmortization,AssetImpairmentCharge,StockBasedCompensation,OtherNonCashItems,...,RetainedEarnings,TotalCapitalization,CommonStockEquity,NetTangibleAssets,WorkingCapital,InvestedCapital,TangibleBookValue,TotalDebt,ShareIssued,OrdinarySharesNumber
0,200.520004,1,4.370000e+09,4.370000e+09,7.943000e+09,1.232000e+09,1.232000e+09,102000000.0,484000000.0,-13000000.0,...,2.788200e+10,6.531600e+10,6.263400e+10,6.201900e+10,2.086800e+10,6.729100e+10,6.201900e+10,9.573000e+09,3.185000e+09,3.185000e+09
1,233.139999,0,3.308000e+09,3.308000e+09,1.878000e+09,1.235000e+09,1.235000e+09,144000000.0,465000000.0,35000000.0,...,1.995400e+10,5.561800e+10,5.346600e+10,5.284100e+10,1.838600e+10,5.717000e+10,5.284100e+10,8.187000e+09,3.179000e+09,3.179000e+09
2,257.260010,0,3.065000e+09,3.065000e+09,2.614000e+09,1.154000e+09,1.154000e+09,167000000.0,445000000.0,44000000.0,...,1.810100e+10,5.163400e+10,5.113000e+10,5.048100e+10,1.628300e+10,5.264900e+10,5.048100e+10,5.811000e+09,3.174000e+09,3.174000e+09
3,202.589996,0,2.513000e+09,2.513000e+09,2.539000e+09,1.046000e+09,1.046000e+09,50000000.0,418000000.0,15000000.0,...,1.539800e+10,4.887100e+10,4.805400e+10,4.747100e+10,1.556100e+10,4.980600e+10,4.747100e+10,5.574000e+09,3.169000e+09,3.169000e+09
4,206.210007,0,3.278000e+09,3.278000e+09,3.707000e+09,9.890000e+08,9.890000e+08,59000000.0,419000000.0,181000000.0,...,1.288500e+10,4.573300e+10,4.470400e+10,4.411100e+10,1.420800e+10,4.674900e+10,4.411100e+10,5.748000e+09,3.164000e+09,3.164000e+09


In [108]:
reg1 = setup(df, train_size=0.80, target = 'y_label', session_id=123)

,Description,Value
0,Session id,123
1,Target,y_label
2,Target type,Binary
3,Original data shape,"(55, 119)"
4,Transformed data shape,"(55, 119)"
5,Transformed train set shape,"(44, 119)"
6,Transformed test set shape,"(11, 119)"
7,Numeric features,118
8,Preprocess,True
9,Imputation type,simple


In [109]:
xgboost=create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6000,0.7500,0.0000,0.0000,0.0000,-0.2500,-0.2500
1,0.8000,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.8000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.8000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.7500,0.3333,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.5000,0.3333,0.0000,0.0000,0.0000,-0.3333,-0.3333
6,0.7500,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.7500,0.3333,0.0000,0.0000,0.0000,0.0000,0.0000
8,1.0000,0.0000,0.0000,0.0000,0.0000,nan,0.0000


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [110]:
knn=create_model('knn')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8000,0.7500,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.8000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.6000,0.3750,0.0000,0.0000,0.0000,-0.2500,-0.2500
3,0.6000,0.2500,0.0000,0.0000,0.0000,-0.2500,-0.2500
4,0.7500,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.7500,0.3333,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.7500,0.1667,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.7500,0.3333,0.0000,0.0000,0.0000,0.0000,0.0000
8,1.0000,0.0000,0.0000,0.0000,0.0000,nan,0.0000


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [111]:
tuned_xgboost = tune_model(xgboost, optimize='Accuracy',choose_better='True',n_iter = 10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8000,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.8000,0.2500,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.8000,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.8000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.7500,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.7500,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.7500,0.3333,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.7500,0.3333,0.0000,0.0000,0.0000,0.0000,0.0000
8,1.0000,0.0000,0.0000,0.0000,0.0000,nan,0.0000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [112]:
tuned_knn = tune_model(knn, optimize='Accuracy',choose_better='True',n_iter = 10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8000,0.2500,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.8000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.8000,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.8000,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.7500,0.1667,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.7500,0.3333,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.7500,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.7500,0.6667,0.0000,0.0000,0.0000,0.0000,0.0000
8,1.0000,0.0000,0.0000,0.0000,0.0000,nan,0.0000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [113]:
pred_holdouts = predict_model(tuned_xgboost)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8182,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000


In [114]:
pred_holdouts = predict_model(tuned_knn)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.8182,0.2222,0.0000,0.0000,0.0000,0.0000,0.0000


In [115]:
plot_model(tuned_xgboost, "feature", save=True, scale = 20)

'Feature Importance.png'

In [116]:
# plot_model(tuned_xgboost, "auc", save=True, scale = 40)